In [49]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
import numpy as np

output_notebook()

Loading BokehJS ...

In [50]:
from bokeh.models import FactorRange
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
#from bokeh.palettes import Spectral6

In [51]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit Counts by Year")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [126]:
from bokeh.palettes import Spectral5
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}


# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

# print ([x for x in zip(data['2015'], data['2016'], data['2017'])])
# sum(zip(data['2015'], data['2016'], data['2017']),())

print ([x for x in zip(x,counts)])
# print ([x for x in zip(data['2015'], data['2016'], data['2017'])])
# print (counts)
# print (x)

from bokeh.transform import factor_cmap

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit Counts by Year")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",

       # use the palette to colormap based on the the x[1:2] values
       fill_color= factor_cmap('x', palette=Spectral5, factors=years, start=1, end=2))

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)


[(('Apples', '2015'), 2), (('Apples', '2016'), 5), (('Apples', '2017'), 3), (('Pears', '2015'), 1), (('Pears', '2016'), 3), (('Pears', '2017'), 2), (('Nectarines', '2015'), 4), (('Nectarines', '2016'), 3), (('Nectarines', '2017'), 4), (('Plums', '2015'), 3), (('Plums', '2016'), 2), (('Plums', '2017'), 4), (('Grapes', '2015'), 2), (('Grapes', '2016'), 4), (('Grapes', '2017'), 5), (('Strawberries', '2015'), 4), (('Strawberries', '2016'), 6), (('Strawberries', '2017'), 3)]


In [129]:
from bokeh.palettes import viridis
networks = ['VGG','Inception', 'Resnet']
confs = ['conf1','conf2','conf3','conf4','conf5','conf6','conf7']

data = np.random.randint(low = 1, high = 1000, size=(len(networks), len(confs)))

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
#x = [ (fruit, year) for fruit in fruits for year in years ]
x = [ (nw,conf) for nw in networks for conf in confs]

print (data.shape)

counts = data.flatten()

print (counts)

source = ColumnDataSource(data=dict(x=x, counts=counts))
from bokeh.transform import factor_cmap

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit Counts by Year")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",

       # use the palette to colormap based on the the x[1:2] values
       fill_color= factor_cmap('x', palette=viridis(len(confs)), factors=confs, start=1))

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

(3, 7)
[287 894 946 301 538 169 968 318 938 736 214 613 905 932 608 779 794 820
 510 127 874]


In [154]:
from bokeh.transform import factor_cmap
from bokeh.models import FactorRange, ColumnDataSource
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.palettes import viridis

import numpy as np

def createMultiLevelPerfCompare(listLevel0, listLevel1, data, title, y_label, plot_height = 250, 
                                tools = None, colorPalletteLevel1 = None):
    ''' Creates a bokeh figure for multi-level perf data (clocks, energy )
        listLevel0 = list of names of Higher level category (e.g. ModelNames)
        listLevel1 = list of names of Lower level category (e.g. TileSize)
        data = 2D numpy array for data arraged in (Highlevel,Lowlevel) format
        title = Title of the figure
        plot_height = Height for generated bokeh plot (Default = 250)
        tools = Set of tools to display along with plot (Defalt = All Bokeh tools)
        colorPalletteLevel1 = a list of colors to use for Lower level category (Default = sample from virdis)
    '''
    
    assert data.shape == (len(listLevel0), len(listLevel1)) 

    # List of tuples for x axis (e.g. [(vgg,conf0),(vgg,conf1)])
    x = [(lvl0,lvl1) for lvl0 in listLevel0 for lvl1 in listLevel1]
    
    # ColumnData for bokeh
    source = ColumnDataSource(data=dict(x=x, counts=data.flatten()))
    
    p = figure(x_range=FactorRange(*x), plot_height=plot_height, title=title, tools=tools)
    
    p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
           fill_color= factor_cmap('x', palette=viridis(len(listLevel1)), factors=listLevel1, start=1))
    
    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None
    p.yaxis.axis_label = y_label
    p.toolbar.logo = None 
    
    return p

In [155]:
#Test createMultiLevelPerfCompare

networks = ['VGG','Inception', 'Resnet']
confs = ['conf1','conf2','conf3','conf4','conf5','conf6','conf7']

#Radom data
data = np.random.randint(low = 1, high = 1000, size=(len(networks), len(confs)))

fig = createMultiLevelPerfCompare(networks, confs, data, "Perf. for multiple runs", "Cycles(in Millions)",
                                  tools= "wheel_zoom,box_zoom,save,reset")

show(fig)